# Stocking rental bikes

![bike rentals](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg/640px-Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg)

You stock bikes for a bike rental company in Austin, ensuring stations have enough bikes for all their riders. You decide to build a model to predict how many riders will start from each station during each hour, capturing patterns in seasonality, time of day, day of the week, etc.

To get started, create a project in GCP and connect to it by running the code cell below. Make sure you have connected the kernel to your GCP account in Settings.

In [1]:
# Set your own project id here
PROJECT_ID = '------------'# a string, like 'kaggle-bigquery-240818'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('model_dataset', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

In [2]:
%load_ext google.cloud.bigquery

## Linear Regression

Your dataset is quite large. BigQuery is especially efficient with large datasets, so you'll use BigQuery-ML (called BQML) to build your model. BQML uses a "linear regression" model when predicting numeric outcomes, like the number of riders.

## 1) Training vs testing

You'll want to test your model on data it hasn't seen before (for reasons described in the Kaggle's [Intro to Machine Learning Micro-Course](https://www.kaggle.com/learn/intro-to-machine-learning). What do you think is a good approach to splitting the data? What data should we use to train, what data should we use for test the model?

In [3]:
# You can write your notes here
table = client.get_table("bigquery-public-data.austin_bikeshare.bikeshare_trips")
client.list_rows(table, max_results=5).to_dataframe()

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900285908,Annual Membership (Austin B-cycle),400,2014-10-26 14:12:00+00:00,2823,Capital Metro HQ - East 5th at Broadway,2544,East 6th & Pedernales St.,10
1,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
2,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
4,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30


## Training data

First, you'll write a query to get the data for model-building. You can use the public Austin bike share dataset from the `bigquery-public-data.austin_bikeshare.bikeshare_trips` table. You predict the number of rides based on the station where the trip starts and the hour when the trip started. Use the `TIMESTAMP_TRUNC` function to truncate the start time to the hour.

## 2) Query the training data

Write the query to retrieve your training data. The fields should be:
1. The start_station_name
2. A time trips start, to the nearest hour. Get this with `TIMESTAMP_TRUNC(start_time, HOUR) as start_hour`
3. The number of rides starting at the station during the hour. Call this `num_rides`.
Select only the data before 2018-01-01 (so we can save data from 2018 as testing data.)

Write your query below:

In [4]:
%%bigquery
SELECT COUNT(*) as num_rides,
TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
start_station_name
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01' 
        GROUP BY start_station_name, start_time
        LIMIT 10 

,num_rides,start_hour,start_station_name
0,1,2014-10-26 14:00:00+00:00,Capital Metro HQ - East 5th at Broadway
1,1,2015-10-02 21:00:00+00:00,Zilker Park West
2,5,2014-10-26 15:00:00+00:00,Toomey Rd @ South Lamar
3,2,2014-10-26 18:00:00+00:00,State Capitol @ 14th & Colorado
4,1,2014-10-26 18:00:00+00:00,Waller & 6th St.
5,1,2014-10-26 20:00:00+00:00,Toomey Rd @ South Lamar
6,1,2014-10-27 15:00:00+00:00,Waller & 6th St.
7,1,2014-10-27 16:00:00+00:00,Main Office
8,3,2014-10-27 17:00:00+00:00,Convention Center/ 3rd & Trinity
9,1,2017-01-29 16:00:00+00:00,Pease Park


You'll want to inspect your data to ensure it looks like what you expect. Run the line below to get a quick view of the data, and feel free to explore it more if you'd like (if you don't know how to do that, the [Pandas micro-course](https://www.kaggle.com/learn/pandas)) might be helpful.

## Model creation

Now it's time to turn this data into a model. You'll use the `CREATE MODEL` statement that has a structure like: 

```sql
CREATE OR REPLACE MODEL`model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS 
-- training data query goes here
SELECT ...
    column_with_labels AS label
    column_with_data_1 
    column_with_data_2
FROM ... 
WHERE ... (Optional)
GROUP BY ... (Optional)
```

The `model_type` and `optimize_strategy` shown here are good parameters to use in general for predicting numeric outcomes with BQML.

**Tip:** Using ```CREATE OR REPLACE MODEL``` rather than just ```CREATE MODEL``` ensures you don't get an error if you want to run this command again without first deleting the model you've created.

## 3) Create and train the model

Below, write your query to create and train a linear regression model on the training data.

Write your query below:

In [5]:
%%bigquery
CREATE OR REPLACE MODEL `model_dataset.bike_model`
OPTIONS(model_type = 'linear_reg', optimize_strategy = 'batch_gradient_descent') as 
SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time < '2018-01-01'
        GROUP BY start_station_name, start_time

""


In [6]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL model_dataset.bike_model)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.242578,1.206606,0.1,13537
1,0,1,1.127036,1.098010,0.2,19562
2,0,2,1.080127,1.052788,0.2,19774
3,0,3,1.058088,1.031621,0.2,19572
4,0,4,1.046263,1.020374,0.2,16863
5,0,5,1.039270,1.013156,0.2,22195


## 4) Model evaluation

Now that you have a model, evaluate it's performance on data from 2018. 


> Note that the ML.EVALUATE function will return different metrics depending on what's appropriate for your specific model. You can just use the regular ML.EVALUATE funciton here. (ROC curves are generally used to evaluate binary problems, not linear regression, so there's no reason to plot one here.)

Write your query below:

In [7]:
%%bigquery
SELECT * FROM ML.EVALUATE(MODEL `model_dataset.bike_model`,(
    SELECT start_station_name,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
    COUNT(*) as label
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
        WHERE start_time >= '2018-01-01'
        GROUP BY start_station_name, start_time
))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.325166,0.137811,0.044132,0.31031,-47.07248,-16.635479


You should see that the r^2 score here is negative. Negative values indicate that the model is worse than just predicting the mean rides for each example.

## 5) Theories for poor performance

Why would your model be doing worse than making the most simple prediction based on historical data?

In [8]:
## Thought question answer here
# The result achieved here is much worse than expected due to the data we're training on.
# We are training the model on acquired data before 2018. And then we are evaluating the data on or after 2018.
# This may be due to the difference in data metrics. There may be a certain spike in business in most days of 2018,
# as compared to that of before 2018.

## 6) Looking at predictions

A good way to figure out where your model is going wrong is to look closer at a small set of predictions. Use your model to predict the number of rides for the 22nd & Pearl station in 2018. Compare the mean values of predicted vs actual riders.

Write your query below:

In [9]:
%%bigquery
SELECT
    AVG(predicted_label), AVG(label)
    FROM ML.PREDICT(MODEL `model_dataset.bike_model`, (
    SELECT COUNT(bikeid) as label,
    TIMESTAMP_TRUNC(start_time, HOUR) as start_hour, 
    start_station_name
        FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
    WHERE DATE(start_time) BETWEEN '2018-01-01' and '2019-01-01'
        and start_station_name LIKE '%22%'
    GROUP BY start_station_name, start_hour))

,f0_,f1_
0,0.600198,3.954864


What you should see here is that the model is underestimating the number of rides by quite a bit. 

## 7) Average daily rides per station

Either something is wrong with the model or something surprising is happening in the 2018 data. 

What could be happening in the data? Write a query to get the average number of riders per station for each year in the dataset and order by the year so you can see the trend. You can use the `EXTRACT` method to get the day and year from the start time timestamp. (You can read up on EXTRACT [in this lesson in the Intro to SQL course](https://www.kaggle.com/dansbecker/order-by)). 

Write your query below:

In [10]:
%%bigquery
WITH RELEVANT as (
    SELECT COUNT(bikeid) as num_rides,
    TIMESTAMP_TRUNC(start_time, YEAR) as year,
    start_station_name
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
    GROUP BY start_station_name, year)
        SELECT EXTRACT(DAY FROM year) as day, 
        start_station_name,
        AVG(num_rides) as avg_rides
            FROM RELEVANT
                GROUP BY start_station_name, year
                ORDER BY year

,day,start_station_name,avg_rides
0,1,Republic Square,90.0
1,1,5th & Bowie,179.0
2,1,2nd & Congress,110.0
3,1,4th & Congress,135.0
4,1,8th & Congress,115.0
5,1,South Congress & James,154.0
6,1,City Hall / Lavaca & 2nd,151.0
7,1,Barton Springs & Riverside,122.0
8,1,South Congress & Elizabeth,74.0
9,1,Capitol Station / Congress & 11th,256.0


## 8) What do your results tell you?

Given the daily average riders per station over the years, does it make sense that the model is failing?

In [11]:
## Thought question answer here
# In the result we can see that there are sometimes a major spike in avg. riders per day and then there is a sudden drop.
# This shows that the model is failing. This may be due to variables on which no linear relationship can be applied or
# Linear regression is not a suitable algorithm to be applied in this problem as a predictive model.